In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import wandb
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from CNN_v1 import CNN
from scripts import load_split_train_valid_test
from torchsummary import summary

In [2]:
# Data path
path = os.path.join(os.path.dirname(os.getcwd()), 'data/')
if torch.cuda.is_available:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [3]:
model = CNN()
summary(model, (3, 224, 224), device="cpu")
model.to(device)

/usr/local/dev/anaconda3/envs/cancer/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
       BatchNorm2d-2         [-1, 16, 224, 224]              32
         LeakyReLU-3         [-1, 16, 224, 224]               0
            Conv2d-4         [-1, 16, 224, 224]           2,320
       BatchNorm2d-5         [-1, 16, 224, 224]              32
         LeakyReLU-6         [-1, 16, 224, 224]               0
         MaxPool2d-7         [-1, 16, 112, 112]               0
           Dropout-8         [-1, 16, 112, 112]               0
             Block-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 32, 112, 112]           4,640
      BatchNorm2d-11         [-1, 32, 112, 112]              64
        LeakyReLU-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 32, 112, 112]           9,248
      BatchNorm2d-14         [-1, 32, 1

CNN(
  (block1): Block(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (leaky_relu1): LeakyReLU(negative_slope=0.1)
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (leaky_relu2): LeakyReLU(negative_slope=0.1)
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.25, inplace=False)
  )
  (block2): Block(
    (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (leaky_relu1): LeakyReLU(negative_slope=0.1)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stat

In [4]:
def sweep():
    config_default = None
    with wandb.init(config=config_default):
        config = wandb.config
        trainloader, testloader, validloader = load_split_train_valid_test(path)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        
        criterion.to(device)

        for epoch in range(config.epochs):
            train_loss = 0
            model.train()
            for i, (images, labels) in enumerate(trainloader):
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
            train_loss /= len(trainloader)
            
            # Evaluate on the validation set
            model.eval()
            true_positive = 0
            true_negative = 0
            false_positive = 0
            false_negative = 0

            valid_loss = 0

            with torch.no_grad():
                for i, (images, labels) in enumerate(validloader):
                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    _, pred = torch.max(outputs.data, 1)
                    pred = pred
                    true_positive += torch.sum(torch.bitwise_and(pred == labels, pred == 0)).item()
                    true_negative += torch.sum(torch.bitwise_and(pred == labels, pred == 1)).item()
                    false_negative += torch.sum(pred == 1).item() - torch.sum(
                        torch.bitwise_and(pred == labels, pred == 1)).item()
                    false_positive += torch.sum(pred == 0).item() - torch.sum(
                        torch.bitwise_and(pred == labels, pred == 0)).item()
                    valid_loss += loss.item()

            valid_f1_score = true_positive / (true_positive + (false_positive + false_negative) / 2)
            valid_accuracy = (true_positive + true_negative) / (
                        true_positive + true_negative + false_positive + false_negative)

            valid_loss /= len(validloader)

            print("Epoch:", epoch + 1, "/", config.epochs)
            wandb.log({
                "Epoch": epoch,
                "train_loss": train_loss,
                "valid_loss": valid_loss,
                "valid_accuracy": valid_accuracy,
                "valid_f1_score": valid_f1_score,
            })

In [5]:
!wandb login

wandb: Currently logged in as: ttra0058 (use `wandb login --relogin` to force relogin)


In [6]:
sweep_config = {
    'name': 'CustomCNN',
    'method': 'grid',
    'parameters': {
        'epochs': {
            'values': [50]
        },
    }
}

In [7]:
sweep_id = wandb.sweep(sweep_config, project="cancer", entity="team17")

Create sweep with ID: 0cbdcbe1
Sweep URL: https://wandb.ai/team17/cancer/sweeps/0cbdcbe1


In [8]:
wandb.agent(sweep_id, sweep)

wandb: Agent Starting Run: b73ng4zq with config:
wandb: 	epochs: 50
wandb: Currently logged in as: ttra0058 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch: 1 / 50
Epoch: 2 / 50
Epoch: 3 / 50
Epoch: 4 / 50
Epoch: 5 / 50
Epoch: 6 / 50
Epoch: 7 / 50
Epoch: 8 / 50
Epoch: 9 / 50
Epoch: 10 / 50
Epoch: 11 / 50
Epoch: 12 / 50
Epoch: 13 / 50
Epoch: 14 / 50
Epoch: 15 / 50
Epoch: 16 / 50
Epoch: 17 / 50
Epoch: 18 / 50
Epoch: 19 / 50
Epoch: 20 / 50
Epoch: 21 / 50
Epoch: 22 / 50
Epoch: 23 / 50
Epoch: 24 / 50
Epoch: 25 / 50
Epoch: 26 / 50
Epoch: 27 / 50
Epoch: 28 / 50
Epoch: 29 / 50
Epoch: 30 / 50
Epoch: 31 / 50
Epoch: 32 / 50
Epoch: 33 / 50
Epoch: 34 / 50
Epoch: 35 / 50
Epoch: 36 / 50
Epoch: 37 / 50
Epoch: 38 / 50
Epoch: 39 / 50
Epoch: 40 / 50
Epoch: 41 / 50
Epoch: 42 / 50
Epoch: 43 / 50
Epoch: 44 / 50
Epoch: 45 / 50
Epoch: 46 / 50
Epoch: 47 / 50
Epoch: 48 / 50
Epoch: 49 / 50
Epoch: 50 / 50


Epoch,49
train_loss,0.504
valid_loss,0.51774
valid_accuracy,0.78774
valid_f1_score,0.72747
_runtime,50687
_timestamp,1629056302
_step,49


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▇▆▆▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▃▂▁▂▁▁▁▁▁▁▁▂
valid_loss,▆▆█▄▅▄▃▂▂▂▃▄▅▂▅▁▃▃▂▃▄▁▃▄▃▆▃▄▂▆▅▄▃▃▃▄▁▃▂▂
valid_accuracy,▂▃▁▄▄▅▅▇▇▇▆▅▅▇▄█▆▆▇▆▄█▅▅▅▃▆▅▇▄▅▅▇▆▇▅█▆▇▇
valid_f1_score,▃▁▅▄▆▆▄▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▆▇▇█▇▇▇█▇█▇█▇█▇
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
